Om de architectuur weer te geven wordt een simple voorbeeld gegeven door een reeks keer twee en gedeelddoor te doen.

De opbouw van de code is hier onder weergegeven:
![opbouw_code](https://github.com/continu-inzicht/toolbox-continu-inzicht/blob/main/docs/assets/opbouw_code.png?raw=true)

Een voorbeeld senario is het inladen van CSV en wegschrijven naar csv:

![voorbeeld csv](https://github.com/continu-inzicht/toolbox-continu-inzicht/blob/main/docs/assets/voorbeeld_config_csv.png?raw=true)

Aan de voorkant ziet dat er zo uit:

In [ ]:
#  importeer de benodigde functies
from pathlib import Path

from continu_inzicht_toolbox.base import Config

Lees de configuratie in:

Dit leest een configuratie bestand `test_config.yaml` in: 

```yaml
WaardesKeerTwee: 
  input: 
      type: csv
      path: 'tests\src\data_sets\test_csv_in.csv'
  output: 
      type: csv
      path: 'tests\src\data_sets\test_csv_out.csv'

...

```

In [ ]:
test_data_sets_path = Path.cwd().parent.parent / "tests" / "src" / "data_sets"
c = Config(config_path=test_data_sets_path / "test_config.yaml")
c.lees_config()

Zet de data adapter klaar en geef deze de configuratie mee 

In [ ]:
from continu_inzicht_toolbox.base import DataAdapter

data = DataAdapter(config=c)


In [ ]:
df_in = data.input("WaardesKeerTwee", **{"csv": {}})
df_in

,objectid,objecttype,parameterid,datetime,value
0,1,measuringstation,1,1726227377000,4.8
1,2,measuringstation,1,1726227377000,5.0
2,3,measuringstation,1,1726227377000,5.2
3,4,measuringstation,1,1726227377000,5.4
4,5,measuringstation,1,1726227377000,5.6


Run de module met CSV

In [ ]:
from continu_inzicht_toolbox.modules import voorbeeld_module

keer_twee = voorbeeld_module.WaardesKeerTwee(data_adapter=data)

In [ ]:
keer_twee.run()

Check de output

In [ ]:
c2 = Config(config_path=test_data_sets_path / "test_output_config.yaml")
c2.lees_config()
data2 = DataAdapter(config=c2)
data2.input("WaardesDelenTwee", **{"csv": {}})
# Met de c2 configuratie doe we het tegen overgestelde, de input is dus de output van de vorige

,Unnamed: 0,objectid,objecttype,parameterid,datetime,value
0,0,1,measuringstation,1,1726227377000,9.6
1,1,2,measuringstation,1,1726227377000,10.0
2,2,3,measuringstation,1,1726227377000,10.4
3,3,4,measuringstation,1,1726227377000,10.8
4,4,5,measuringstation,1,1726227377000,11.2


de `values` waardes uit de vorige table zijn vermenigvuldigd met twee

Herhaal met database:

![voorbeeld_config_db](https://github.com/continu-inzicht/toolbox-continu-inzicht/blob/main/docs/assets/voorbeeld_config_db.png?raw=true)

In [ ]:
df_in = data.input(
    "WaardesDelenTwee",
    **{"postgresql_database": {"table": "data", "schema": "citoolbox_schema"}},
)
df_in

,objectid,objecttype,parameterid,datetime,value
0,1,measuringstation,1,1726227377000,0.3


In [ ]:
delen_twee = voorbeeld_module.WaardesDelenTwee(data_adapter=data)

In [ ]:
delen_twee.run()

check

In [ ]:
data2.input(
    "WaardesKeerTwee",
    **{"postgresql_database": {"table": "data", "schema": "citoolbox_schema"}},
)

,objectid,objecttype,parameterid,datetime,value
0,1,measuringstation,1,1726227377000,0.15


_Maak de aanpassingen in de testdatabase ongedaan_

In [ ]:
delen_twee_ongedaan = voorbeeld_module.WaardesKeerTwee(data_adapter=data2)
delen_twee_ongedaan.run()

In [ ]:
data2.input(
    "WaardesKeerTwee",
    **{"postgresql_database": {"table": "data", "schema": "citoolbox_schema"}},
)

,objectid,objecttype,parameterid,datetime,value
0,1,measuringstation,1,1726227377000,0.3
